## 使用随机森林构建投资组合

https://www.ricequant.com/community/topic/38302/

随机森林能对高维数据集进行快速的训练和预测，并保持较高的精度，同时减少过拟合的风险。以下我们用它来构建投资组合。

### 基本思路

常见的思路是把随机森林模型作为一个股票的分类器。

我们在这里把股票未来一个月的收益作为预测目标，把未来一个月收益率高的样本看做正例，收益率低的样本看做负例，在本文中我们将10%作为划分比例，读者可以用自己的方法来定义正例和负例。在构建组合时，我们把股票的特征输入已经训练好的模型中，模型就会输出每一条样本的属于正例的概率，然后将这个概率作为选股因子。

### 数据说明

本文所使用的原始数据集包含2008年1月至12月每个月月末的横截面数据，字段包括月末日期、股票代码、未来收益率、个股特征（包含停牌状态、ST状态、上市天数）等用于判断是否符合交易条件的特征和EP、ROE等参与训练用于选股的特征。由于本文使用本地数据集，读者如果有复现需要，可以根据文末的代码实例构建一个类似的数据集。

在输入模型进行训练之前需要先对数据集进行预处理。数据集包含不满足可交易条件的样本，需要将这些样本剔除。被定义为可交易的股票须同时满足以下条件：1）非ST；2）非停牌；3）上市时间超过3个月。

forward_return：未来收益率
is_suspend：停牌状态
is_ST：ST状态
days_listed:上市天数
等用于判断是否符合交易条件的特征

EP：
ROE：



In [ ]:
import os
import pickle
from collections import OrderedDict
import pandas as pd
import sklearn as skl
from sklearn.ensemble import RandomForestClassifier
from sklearn import grid_search
import matplotlib.pyplot as plt

raw_data = pd.read_csv('raw_data.csv',parse_dates=['date'],index_col=['date'])

### 模型训练

我们使用sklearn中的随机森林实现。注意，sklearn输出的数据结构多为numpy.array，而我们项目主要处理的数据结构是pandas.Series和pandas.DataFrame。因此我们需要定义以下两个wrapper函数，将sklearn的输出转化为我们所需的数据结构。

#### 定义两个函数

make_model：输入预测日，所有历史数据，创建并训练模型。在实现上，它接受了一些主要super parameter用于创建随机森林模型，在内部调用get_training_dataset得到训练集，调用model.fit训练模型。

make_factor：我们最终的目标是通过机器学习模型得到一个选股因子，因此我们定义一个函数，在预测日，根据模型进行预测，将模型的输出作为因子。在实现上，它在内部调用model.predict_proba方法，并将其转换为与我们项目一致的数据结构。

In [1]:



def make_model(rebalance_date, data, n_estimators=50, max_depth=9):
    model = RandomForestClassifier(n_estimators=n_estimators,bmax_depth=max_depth)
    X, y = get_training_dataset(rebalance_date, data)
    model.fit(X, y)
    return model

def make_factor(rebalance_date, model, data):
    features = get_prediction_dataset(rebalance_date, data)  values =  model.predict_proba(features)[:,1]
    index = pd.MultiIndex.from_product([[rebalance_date], features.index], names=['date', 'symbol'])   
    res = pd.Series(values, index)
    return res

SyntaxError: invalid syntax (<ipython-input-1-a2187020daff>, line 8)

本文以48个月为窗口进行滚动训练。初始化时需要用掉的样本数一般称为burn in。在我们的例子里，第一个模型需用掉48期，故BURNIN_PERIODS = 48。模型训练一般在换仓日进行，我们先定义一个换仓日列表rebalance_dates用于循环，然后循环训练集以获得48期的选股因子数据。

In [ ]:
BURNIN_PERIODS = 48
rebalance_dates = raw_data.index.get_level_values('date').unique()[BURNIN_PERIODS: ]
factor_list = []
model_dict = {}
for date in rebalance_dates:
    print('Training model for {}'.format(date))
    model = makfactor.to_frame('factor').to_csv('rf_rolling_factor_data.csv')e_model(date, raw_data)
    model_dict[date] = model
    factor = make_factor(date, model, raw_data)
    factor_list.append(factor)
factor = pd.concat(factor_list)

### 特征重要性分析

如果模型支持计算特征重要性，sklearn提供了model.feature_importances_这一API来获取。但它返回的是numpy array，没有对应特征的名称，我们需要自己加上去。 在这里我们展示几个典型的时点，如2015年牛市期间(2015年3月)。

In [ ]:
def get_feature_importances(model, names):
    value = model.feature_importances_
    return pd.Series(value, index=names)

model_201503 = model_dict[pd.Timestamp('2015-03-31')]
importances_201503 = get_feature_importances(model_201503, feature_cols)
importances_201503.sort_values(ascending=False).plot(kind='bar', figsize=(18, 6), title='Feature Importances on 2015-03')

同理可以进行2015年股灾后（2016年3月)和2017年熊市期间（2017年8月）的特征重要性分析。

### 超参数优化

我们可以使用超参数优化对每一种参数组合进行训练和交叉验证（Cross Validation)，以得到最优模型和最优参数。



超参数优化的一个常见场景是网格搜索，即对每个待优化参数指定一组待测试的值，然后穷举每一种参数组合。sklearn提供了GridSearchCV API来进行网格搜索。GridSearchCV继承自Estimator，它也支持fit、predict等API。



GridSearchCV有两个主要参数：estimator：待优化的模型；param_grid：参数网格。参数网格通过dict来指定。它的key为待优化参数的名称，应和待优化模型的参数名称一致。它的value为待搜索的参数列表。如本例中，我们针对n_estimators（弱学习器的最大迭代次数）和max_depth(树的最大深度）两个参数进行优化。n_estimators的搜索空间为[30, 40, 50]，max_depth的搜索空间为range(2, 9)。未在param_grid中指定的参数保持为默认参数，不进行优化。

In [ ]:
param_grid = {'n_estimators': [30, 40, 50], 'max_depth': range(2, 10)}
model = skl.model_selection.GridSearchCV(RandomForestClassifier(), param_grid, cv=5)
X, y = get_training_dataset(pd.Timestamp('2017-08-31'), raw_data)
model.fit(X, y)

训练完成后，可以通过best_estimator_和best_params查看最优模型和最优参数,并进行优化后的参数比较。



sklearn中随机森林模型的文档详见scikit-learn官方文档（中文版）中的RandomForestClassifier 和 RandomForestRegressor 类。注意sklearn随机森林分类器的实现是取每个分类器预测概率的平均值，而不是让每个分类器对类别进行投票。

### 构建目标组合

机器学习模型负责对每一条样本进行预测得到，得到每个个股的“分数”。而如何根据“分数”构建目标持仓，是另一个独立的过程。本文将展示两种方式。

01

方法一：分位数组合

以下使用的构建分位数（quantile）组合的方法是每一期买入分数位于当前股票池前10%的股票。在这种方法下，每一期的持仓数量会略有不同，随着当期可投资股票的数量变化而变化。按分位数构建筑的方法还会用来评价因子的单调性，在这里我们同时计算十个分位数组合，并考察其单调性。



在实际操作中，我们使用pd.qcut将指定因子对每个横截面Series按分位数分组，返回每个个股每一天所属的组合，然后我们从分组构建目标组合。

In [ ]:
def bucketize_by_quantile(factor, quantiles=10):
    quantile_labels = list(range(1, quantiles + 1))
    quantile_groups = (factor.groupby(level='date', group_keys=False).apply(pd.qcut,q=quantiles,labels=quantile_labels))
    return quantile_groups

def get_portfolios_from_quantile_groups(quantile_groups, quantile):
    selected = quantile_groups[quantile_groups == quantile]
    def build_portfolio(x):
        # 等权组合, 权重为 1/len(x), index为股票代码---x.index的第二层
        return pd.Series(1 / len(x), index=x.index.get_level_values('symbol'))
    portfolios = OrderedDict()
    for date, x in selected.groupby(level='date'):
        portfolios[date] = build_portfolio(x)
    return portfolios

02

计算净值

计算组合的累计收益，一个简单的算法是：直接将个股的日度收益加权平均算出组合的日度收益，再累乘得到整个组合的累计收益。但这个算法对每一天都采用了相同的权重，相当于每一天都在rebalance。而实际各持仓股票的权重每天都会波动，只有在调仓日才等于目标权重。因此这种算法只能看作实际收益的近似值。



在这里我们采取更为精确的做法：计算对各持仓股票的累计收益，而不是日度收益进行加权平均。相当于期初将资金按权重分配到每个股票，期末各持仓股票的价值等于权重乘以累计收益，整个组合在期末的价值等于各持仓股票价值之和。但每一次调仓持仓股票及目标权重都可能发生变化，因此需要分别计算每个持仓期的组合净值，最后再将各期净值按时间顺序连接起来。而对净值的连接，相当于用上一时间段最后一天的净值作为初始资金，再投资到下一期。



以上计算方法可以总结为以下流程：使用net_value作为计算组合净值的主函数，以换仓日期、各换仓日的目标组合、所有个股日度收益为输入，循环调用period_net_value计算各个持仓周期的净值，最后调用merge_period_net_value按照再投资的方式将各个阶段的净值连接为整个回测区间的净值。

In [ ]:
def net_value(rebalance_dates, portfolio_weights, daily_returns):
    net_value_list = []
    for start, end in zip(rebalance_dates[:-1], rebalance_dates[1:]):
        weights = portfolio_weights[start]  
        holding_period = (daily_returns.index >= start) & (daily_returns.index < end)# 每个持仓周期为左闭右开区间,以确保一个交易日属于且只属于一个持仓周期
        assets_held = weights.index
        rtn = daily_returns.loc[holding_period, assets_held].fillna(0)
        net_value_list.append(period_net_value(rtn, weights))
    res = merge_period_net_value(net_value_list)
    return res

def period_net_value(daily_returns, weights):
    asset_net_value = (1 + daily_returns).cumprod()
    normalized_weights = weights / weights.sum()
    portf_net_value = asset_net_value.dot(normalized_weights)
    return portf_net_value

def merge_period_net_value(period_net_values):
    net_value_list = []
    init_capital = 1
    for nv in period_net_values:
         nv *= init_capital
         net_value_list.append(nv)
         init_capital = nv.iat[-1] # 下一段净值的初始资金是上一段最后一天的值
    res = pd.concat(net_value_list)
    res.iloc[0] = 1 # 整个回测期第一天的净值一定是1
    return res

In [ ]:
然后计算每个组合的净值：

In [ ]:
daily_returns = dtl.get_daily_return()
QUANTILE_GROUPS = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
quantile_portf_net_values = OrderedDict()
for group in QUANTILE_GROUPS:
    portfolio_weights = get_portfolios_from_quantile_groups(quantile_groups, group)
    rebalance_dates = list(portfolio_weights.keys())
    nv = net_value(rebalance_dates, portfolio_weights, daily_returns)
    quantile_portf_net_values[group] = nv
pd.concat(quantile_portf_net_values, axis=1).plot(figsize=(18, 6), grid=True)

In [ ]:
第10组的累计净值如下：

In [ ]:
quantile_portf_net_values[10].plot(figsize=(18, 6), grid=True, color='#00BFFF')

03

方式二：取固定数量股票构建组合

另一种常见并且更加简单的组合构建方法是每一期取分数最高的若干个数量股票作为目标组合，使用pandas.Series.nlargest可以方便地得到一个Series内最大的N个值。这里以取每一期取最高分的50只股票为例。

In [ ]:
def top_N_portfolio(factor, N):
    def build_portfolio(x):
        # 等权组合, 权重为 1/len(x), index为股票代码---x.index的第二层
        return pd.Series(1 / len(x), index=x.index.get_level_values('symbol'))
    portfolios = OrderedDict()
    for date, x in factor.groupby(level='date'):
        portfolios[date] = build_portfolio(x.nlargest(N))
    return portfolios
N = 50
top_50_portfolios = top_N_portfolio(factor, N)

In [ ]:
使用上一部分中的计算方法计算净值，这一组合的累计净值如下：

In [ ]:
rebalance_dates = list(portfolio_weights.keys())
nv = net_value(rebalance_dates, top_50_portfolios, daily_returns)
nv.plot(figsize=(18, 6), grid=True, color='#00BFFF')